In [ ]:
! pip install git+https://github.com/huggingface/diffusers transformers accelerate

  Cloning https://github.com/huggingface/diffusers to /tmp/pip-req-build-8izk_sny
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/diffusers /tmp/pip-req-build-8izk_sny
  Resolved https://github.com/huggingface/diffusers to commit 5873377a660dac60a6bd86ef9b4fdfc385305977
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 69.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.1 MB/s eta 0:00:0

**Load the Model and Scheduler**

In [ ]:
import torch
import numpy as np
from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler
from diffusers.utils import export_to_video
from IPython.display import Video

# Charger le pipeline Text-to-Video
pipe = DiffusionPipeline.from_pretrained(
    "damo-vilab/text-to-video-ms-1.7b",
    torch_dtype=torch.float16,
    variant="fp16"
).to("cuda")

pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

**Optimize for GPU Usage**

In [ ]:
# Optimisation mémoire GPU
pipe.enable_model_cpu_offload()
pipe.enable_attention_slicing()  # VAE slicing n'est pas supporté ic

**Define Prompt and Generate Frames**

In [ ]:
# Générer la vidéo à partir du texte
prompt = (
    "A luxurious face serum bottle made of frosted glass with a gold metallic cap, "
    "placed on a marble surface surrounded by drops of water and soft white flowers. "
    "The camera slowly zooms in with cinematic lighting and shallow depth of field. "
    "As the bottle rotates slowly, golden serum drops fall in slow motion beside it. "
    "Elegant floating text appears: 'Hydrate. Illuminate. Transform.' "
    "Soft mist flows in the background, creating a dreamy atmosphere. "
    "High-end beauty product commercial style, smooth camera movement, 4K quality, reflective surfaces, "
    "with gentle background sparkle and soft piano music ambiance."
)

output = pipe(prompt, num_inference_steps=50, num_frames=24)
video_frames = output.frames[0]  # Supprimer la dimension batch

# Convertir les frames au format vidéo
video_array = (video_frames * 255).astype(np.uint8)
video_path = export_to_video(video_array)

# Afficher la vidéo dans le notebook
Video(video_path, embed=True)


Token indices sequence length is longer than the specified maximum sequence length for this model (118 > 77). Running this sequence through the model will result in indexing errors
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['mist flows in the background , creating a dreamy atmosphere . high - end beauty product commercial style , smooth camera movement , 4 k quality , reflective surfaces , with gentle background sparkle and soft piano music ambiance .']


  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
import torch
import numpy as np
from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler
from diffusers.utils import export_to_video
from IPython.display import Video

# Charger le pipeline
pipe = DiffusionPipeline.from_pretrained(
    "damo-vilab/text-to-video-ms-1.7b",
    torch_dtype=torch.float16,
    variant="fp16"
).to("cuda")

pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)

# Optimisation GPU
pipe.enable_model_cpu_offload()
pipe.enable_attention_slicing()

# Prompt
prompt = (
    "A luxurious face serum bottle made of frosted glass with a gold metallic cap, "
    "placed on a marble surface surrounded by drops of water and soft white flowers. "
    "The camera slowly zooms in with cinematic lighting and shallow depth of field. "
    "As the bottle rotates slowly, golden serum drops fall in slow motion beside it. "
    "Elegant floating text appears: 'Hydrate. Illuminate. Transform.' "
    "Soft mist flows in the background, creating a dreamy atmosphere. "
    "High-end beauty product commercial style, smooth camera movement, 4K quality, reflective surfaces, "
    "with gentle background sparkle and soft piano music ambiance."
)
# Génération
output = pipe(prompt, num_inference_steps=50, num_frames=24)
video_frames = output.frames[0]  # (24, 256, 256, 3), float32 entre 0 et 1

# Correction : clamp + conversion
video_array = np.clip(video_frames * 255, 0, 255).astype(np.uint8)

# Export
video_path = export_to_video(video_array)

# Affichage
Video(video_path, embed=True)


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (118 > 77). Running this sequence through the model will result in indexing errors
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['mist flows in the background , creating a dreamy atmosphere . high - end beauty product commercial style , smooth camera movement , 4 k quality , reflective surfaces , with gentle background sparkle and soft piano music ambiance .']


  0%|          | 0/50 [00:00<?, ?it/s]